## U-48800

### Molecular dynamics simulation

#### Environment setup

In [ ]:
# Define imports
import re
import os
import sys
import subprocess
print("Python version: ", sys. version)

import numpy as np
print("Numpy version: ", np.__version__)

import seaborn as sns
print("Seaborn version: ", sns.__version__)

import matplotlib
import matplotlib.pyplot as plt
print("Matplotlib version: ", matplotlib.__version__)

In [60]:
# Set the working directory and change folder to it
working_dir = '/home/familia/research/synthetic_opioids'
os.chdir(working_dir)

# Import glx auxiliary functions
from gmx import gmx, xvg

In [61]:
# Set molecule path, molecule filename and molecular dynamics directory path
compound = "U48800"
mdscmode = "rest"
solvent = "methanol"

# Parameters
replicas = 10
tmin = 293.15
tmax = 573.15

In [62]:
# Set paths
basepath = "%s/%s/%s" % (compound, mdscmode, solvent)
parapath = "%s/%s/parameters" % (compound, mdscmode)
origpath = "%s/%s/original/%s" % (compound, mdscmode, solvent)
libppath = "%s/%s/ligpargen" % (compound, mdscmode)

In [63]:
# Set temperature values for replica exchange molecular dynamics (https://www.plumed.org/doc-v2.9/user-doc/html/hrex.html)
# Adjusting the previous code to round the temperatures to 2 decimal places
temperature_value = [round(tmin * np.exp(i * np.log(tmax / tmin) / (replicas - 1)), 2) for i in range(replicas)]

# Print the list
print("Temperatures:", temperature_value)

# Compute lambda values
lambda_value = [1.0000] + [round(x, 3) for x in temperature_value[0] / temperature_value[1:]]
print("Lambda values:", lambda_value)

# Compute kappa values
kappa_value = [round(0.916248 + 0.0002857 * x, 3) for x in temperature_value]
print("Kappa values:", kappa_value)

Temperatures: [293.15, 315.82, 340.25, 366.56, 394.91, 425.46, 458.36, 493.81, 532.0, 573.15]
Lambda values: [1.0, 0.928, 0.862, 0.8, 0.742, 0.689, 0.64, 0.594, 0.551, 0.511]
Kappa values: [1.0, 1.006, 1.013, 1.021, 1.029, 1.038, 1.047, 1.057, 1.068, 1.08]


In [64]:
# Create the molecular dynamics directory if it does not exist
os.makedirs(basepath, exist_ok = True)

# Create the directory to store all original input files for the simulation
os.makedirs(origpath, exist_ok = True)

# Create the molecular dynamics parameters directory if it does not exist
os.makedirs(parapath, exist_ok = True)

# Create the directory ligpargen output if it does not exist
os.makedirs(libppath, exist_ok = True)

# Delete all files in the molecular dynamics directory
files = [file for file in os.listdir(basepath) if os.path.isfile(os.path.join(basepath, file))]
for file in files:
    os.remove("%s/%s" % (basepath, file))

#### Simulation setup

LigParGen (https://traken.chem.yale.edu/ligpargen/ (https://doi.org/10.1073/pnas.0408037102, https://doi.org/10.1021/acs.jpcb.7b00272, https://doi.org/10.1093/nar/gkx312)) was used to generate the force field (OPLS-AA) parameters for U-48800 based on the PDB file. This resulted in three files, the force field definition file (U48800.itp), the topology file (U48800.top), and the gromacs structure file (U48800.gro), required for molecular dynamics simulations with gromacs.
Note that the U48800.top file was edited to have the information from the forcefield for the solute (U48800.itp) and water tip4p.itp from OPLS-AA.

In [ ]:
# Copy the molecule force field and structure files to the molecular dynamics directory from ligpargen output zip file
#os.system("cp %s/%s.itp %s" % (libppath, compound, origpath))
#os.system("cp %s/%s.gro %s" % (libppath, compound, origpath))
#os.system("cp %s/%s.top %s" % (libppath, compound, origpath))

In [65]:
# Copy the required files to the molecular dynamics directory
os.system("cp %s/*.pdb %s/" % (compound, origpath))
os.system("cp %s/* %s/" % (origpath, basepath))

0

In [66]:
# Define the box shape and size
gmx(
    executable = "editconf",
    arguments = [
        "-bt", "cubic",                # Box type
        "-d", "1.0",                   # Distance between solute and box edge
        "-c"                           # Center the molecule
    ],
    inputs = {
        "-f": "%s.gro" % compound      # Input structure file
    },
    outputs = {
        "-o": "%s.gro" % compound      # Output structure file
    },
    path = "%s" % (basepath)
)

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 46 atoms
Volume: 1 nm^3, corresponds to roughly 400 electrons
No velocities found
    system size :  0.695  0.964  0.901 (nm)
    diameter    :  1.155               (nm)
    center      :  0.050  0.135 -0.031 (nm)
    box vectors :  1.000  1.000  1.000 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :   1.00               (nm^3)
    shift       :  1.528  1.443  1.608 (nm)
new center      :  1.577  1.577  1.577 (nm)
new box vectors :  3.155  3.155  3.155 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :  31.40               (nm^3)

            :-) GROMACS - gmx editconf, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol
Command line:
  gmx editconf -bt cubic -d 1.0 -c -f U48800.gro -o U48800.gro


Back 

In [67]:
# Add methanol molecules to the system
gmx(
    executable = "insert-molecules",
    arguments = [
        "-nmol", "1000",               # Number of molecules
        "-try", "100"                   # Number of attempts
    ],
    inputs = {
        "-f": "%s.gro" % compound,     # Input structure file
        "-ci":  "methanol.gro"      # Topology file
    },
    outputs = {
        "-o": "%s.gro" % compound       # Output structure file
    },
    path = "%s" % (basepath)
)


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------


        :-) GROMACS - gmx insert-molecules, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/r

#### Minimisation

Edit the last lines of U48800.top to match:

```
[ system ]
SOP in Methanol

[ molecules ]
;molecule name   nr.
SOP              1
MET              728 ; Number of methanol molecules added
```

In [68]:
# Prepare the system for minimization
ensemble = "min"

# Copy the parameter file to the directory
os.system("cp %s/%s.mdp %s" % (parapath, ensemble, basepath))

# Remove the flexible preprocessing directive
os.system("sed -i '/^define                  = -DFLEXIBLE    ; defines to pass to the preprocessor/d' %s/%s.mdp" % (basepath, ensemble))


gmx(
    executable = "grompp",
    arguments = [
        "-maxwarn", "2"                    # Maximum number of warnings (there is a warning relative to charge)
    ],
    inputs = {
        "-f": "%s.mdp" % ensemble,  # Minimization parameters file
        "-c": "%s.gro" % compound,         # Input structure file
        "-p": "%s.top" % compound          # Topology file
    },
    outputs = {
        "-po": "%s.mdp" % ensemble,                  # Output parameters file
        "-pp": "%s.top" % ensemble,                  # Output topology file
        "-o" : "%s.tpr" % ensemble                  # Output portable xdr run input file
    },
    path = "%s" % (basepath)
)

Setting the LD random seed to -1191403801

Generated 369370 of the 369370 non-bonded parameter combinations

Generated 369370 of the 369370 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'SOP'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MET'

turning H bonds into constraints...

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Analysing residue names:
There are:     1      Other residues
There are:   728    Protein residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing Protein...

The largest distance between excluded atoms is 0.403 nm between atom 20 and 22
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 2

In [69]:
# Run the simulation
gmx(
    executable = "mdrun",
    ensemble = ensemble,
    arguments = [
        "-v",                                   # Verbose output
        "-deffnm", ensemble,                    # Input and output file name prefix
        "-dlb", "auto"                          # Dynamic load balancing auto
    ],
    inputs = {},
    outputs = {}, 
    name = compound,
    path = basepath, 
    wait = True
)

Submitted batch job 399


In [70]:
# Generate the pressure xvg file
measure = 11 # Potential energy

gmx(
    executable = "energy",
    arguments = [],
    inputs = {
        "-f": "%s.edr" % ensemble
    },
    outputs = {
        "-o": "%s.xvg" % ensemble
    },
    stdin = "%d 0\n" % measure, 
    path = "%s" % (basepath)
)

# Generate the display the corresponding plots
xvg(title = "Energy minimization",
    path = basepath,
    ensemble = ensemble,
    xlabel = "Energy minimization step",
    ylabel = "Energy ($kJ\\cdot mol^{-1}$)",
    label = "Potential Energy",
    movavg = False
    )


Statistics over 10001 steps [ 0.0000 through 10000.0000 ps ], 1 data sets
All statistics are over 7918 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                  -9267.55       2700    16453.1   -17152.3  (kJ/mol)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol
Command line:
  gmx energy -f min.edr -o min.xvg

Opened min.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-Imp.-Dih.

In [71]:
# Backup all progress
os.makedirs("%s/../%s.back" % (basepath, solvent), exist_ok = True)
os.makedirs("%s/../%s.back/%s" % (basepath, solvent, ensemble), exist_ok = True)
os.system("cp -R %s/* %s/../%s.back/%s/" % (basepath, basepath, solvent, ensemble))

0

#### NVT Equilibration

In [72]:
# Define the ensemble
presembl = "min" # Previous ensemble
ensemble = "nvt" # Ensemble

# Copy the parameter file to the work directory
os.system("cp %s/%s.mdp %s" % (parapath, ensemble, basepath))
    
gmx(
    executable = "grompp",
    arguments = [
        "-maxwarn", "2"                              # Maximum number of warnings (there is a warning relative to charge)
    ],
    inputs = {
        "-f": "%s.mdp" % ensemble,                   # Minimization parameters file
        "-c": "%s.gro" % presembl,                   # Input structure file
        "-p": "%s.top" % presembl                    # Topology file
    },
    outputs = {
        "-po": "%s.mdp" % ensemble,                  # Output parameters file
        "-pp": "%s.top" % ensemble,                  # Output topology file
        "-o" : "%s.tpr" % ensemble                   # Output portable xdr run input file
    },
    path = "%s" % (basepath)
)

Setting the LD random seed to -21692565

Generated 369370 of the 369370 non-bonded parameter combinations

Generated 369370 of the 369370 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'SOP'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MET'

turning H bonds into constraints...

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------


Setting gen_seed to -67142145

Velocities were taken from a Maxwell distribution at 293.15 K
Analysing residue names:
There are:     1      Other residues
There are:   728    Protein residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing Protein...

The largest distance between excluded atoms is 0.402 nm be

In [73]:
# Run the simulation
gmx(
    executable = "mdrun",
    ensemble = ensemble,
    arguments = [
        "-v",                                   # Verbose output
        "-deffnm", ensemble,                    # Input and output file name prefix
        "-dlb", "auto"                          # Dynamic load balancing auto
    ],
    inputs = {},
    outputs = {}, 
    name = compound,
    path = basepath, 
    wait = True
)

Submitted batch job 400


In [74]:
#Generate the temperature xvg file
measure = 15     # Temperature

# Compute the the selected parameter from the ensemble simulation
gmx(
    executable = "energy",
    arguments = [],
    inputs = {
        "-f": "%s.edr" % ensemble
    },
    outputs = {
        "-o": "%s.xvg" % ensemble
    },
    stdin = "%d 0\n" % measure, 
    path = "%s" % (basepath)
)

# Generate the display the corresponding plots
xvg(title = "NVT Equilibration",
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Temperature (K)",
    label = "Temperature",
    movavg = 10
    )


Statistics over 100001 steps [ 0.0000 through 100.0000 ps ], 1 data sets
All statistics are over 1001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 292.589       0.15    5.38851   0.182825  (K)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol
Command line:
  gmx energy -f nvt.edr -o nvt.xvg

Opened nvt.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-Imp.-Dih.
  5  LJ-14    

In [75]:
# Backup all progress
os.makedirs("%s/../%s.back" % (basepath, solvent), exist_ok = True)
os.makedirs("%s/../%s.back/%s" % (basepath, solvent, ensemble), exist_ok = True)
os.system("cp -R %s/* %s/../%s.back/%s/" % (basepath, basepath, solvent, ensemble))

0

#### NPT Equilibration

In [76]:
# Define the ensemble
presembl = "nvt" # Previous ensemble
ensemble = "npt" # Ensemble

# Copy the parameter file to the work directory
os.system("cp %s/%s.mdp %s" % (parapath, ensemble, basepath))

# Change the compressibility of the system to match that of methanol
os.system("sed -i 's/compressibility         = 4.96e-5/compressibility         = 1.2e-5/g' %s/%s.mdp" % (basepath, ensemble))
    
gmx(
    executable = "grompp",
    arguments = [
        "-maxwarn", "2"                              # Maximum number of warnings (there is a warning relative to charge)
    ],
    inputs = {
        "-f": "%s.mdp" % ensemble,                   # Minimization parameters file
        "-c": "%s.gro" % presembl,                   # Input structure file
        "-p": "%s.top" % presembl                    # Topology file
    },
    outputs = {
        "-po": "%s.mdp" % ensemble,                  # Output parameters file
        "-pp": "%s.top" % ensemble,                  # Output topology file
        "-o" : "%s.tpr" % ensemble                   # Output portable xdr run input file
    },
    path = "%s" % (basepath)
)

Setting the LD random seed to -17901063

Generated 369370 of the 369370 non-bonded parameter combinations

Generated 369370 of the 369370 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'SOP'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MET'

turning H bonds into constraints...

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Analysing residue names:
There are:     1      Other residues
There are:   728    Protein residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing Protein...

The largest distance between excluded atoms is 0.409 nm between atom 20 and 22

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 293.15 K

In [77]:
# Run the simulation
gmx(
    executable = "mdrun",
    ensemble = ensemble,
    arguments = [
        "-v",                                   # Verbose output
        "-deffnm", ensemble,                    # Input and output file name prefix
        "-dlb", "auto"                          # Dynamic load balancing auto
    ],
    inputs = {},
    outputs = {}, 
    name = compound,
    path = basepath, 
    wait = True
)

Submitted batch job 401


In [ ]:
ensemble = "npt" # Ensemble

In [85]:
# Generate the pressure xvg file
measure = 17     # Pressure
sufix = "_p"

# Compute the the selected parameter from the ensemble simulation
gmx(
    executable = "energy",
    arguments = [],
    inputs = {
        "-f": "%s.edr" % ensemble
    },
    outputs = {
        "-o": "%s%s.xvg" % (ensemble, sufix)
    },
    stdin = "%d 0\n" % measure, 
    path = "%s" % (basepath)
)

# Generate the display the corresponding plots
xvg(title = "NPT Equilibration",
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Pressure (bar)",
    label = "Pressure",
    sufix = sufix,
    movavg = 10
    )

In [79]:
# Generate the density xvg file
measure = 23     # Density
sufix = "_d"

# Compute the the selected parameter from the ensemble simulation
gmx(
    executable = "energy",
    arguments = [],
    inputs = {
        "-f": "%s.edr" % ensemble
    },
    outputs = {
        "-o": "%s%s.xvg" % (ensemble, sufix)
    },
    stdin = "%d 0\n" % measure, 
    path = "%s" % (basepath)
)

# Generate the display the corresponding plots
xvg(title = "NPT Equilibration",
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Density ($Kg\\cdot m^{-3}$)",
    label = "Density",
    sufix = sufix,
    movavg = 10
    )


Statistics over 100001 steps [ 0.0000 through 100.0000 ps ], 1 data sets
All statistics are over 1001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Density                     805.801         18    48.6232   -104.312  (kg/m^3)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol
Command line:
  gmx energy -f npt.edr -o npt_d.xvg

Opened npt.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-Imp.-Dih.
  5  LJ

In [80]:
# Backup all progress
os.makedirs("%s/../%s.back" % (basepath, solvent), exist_ok = True)
os.makedirs("%s/../%s.back/%s" % (basepath, solvent, ensemble), exist_ok = True)
os.system("cp -R %s/* %s/../%s.back/%s/" % (basepath, basepath, solvent, ensemble))

0

#### REST2 setup

In [81]:
# Prepare the system for replica exchange with solute tempering (NPT)
presembl = "npt" # Previous ensemble
ensemble = "mds" # Ensemble

# Copy the parameter file to the directory
os.system("cp %s/%s.mdp %s" % (parapath, ensemble, basepath))

# Change the compressibility of the system to match that of methanol
os.system("sed -i 's/compressibility         = 4.96e-5/compressibility         = 1.2e-5/g' %s/%s.mdp" % (basepath, ensemble))

# Compile a regular expression to match lines with the specified pattern
pattern = re.compile(r'^\s*\d+.*SOP')

# Process the file
with open("%s/%s.top"%(basepath, presembl), 'r') as infile, open("%s/%s.%s.top"%(basepath, presembl, mdscmode), 'w') as outfile:
    for line in infile:
        # Check if the line matches the pattern
        if pattern.search(line):
            # Perform the replacement operation
            modified_line = re.sub(r'(\s*\d*\s*)([a-z]*_\d+)(\s)(.*)', r'\1\2_\4', line)
            outfile.write(modified_line)
        else:
            outfile.write(line)

In [82]:
# Generate scaled topologies
# Create the directories for replicas if they do not exist
for i in range(1, replicas + 1):
    os.makedirs("%s/replica%02d" % (basepath, i), exist_ok = True)

for i in range(1, replicas + 1):
    print("plumed partial_tempering %0.3f < %s.%s.top > replica%02d/%s.top" % (lambda_value[i - 1], presembl, mdscmode, i, presembl))
    subprocess.run("plumed partial_tempering %0.3f < %s.%s.top > replica%02d/%s.top" % (lambda_value[i - 1], presembl, mdscmode, i, presembl), shell = True, cwd = basepath)

plumed partial_tempering 1.000 < npt.rest.top > replica01/npt.top
plumed partial_tempering 0.928 < npt.rest.top > replica02/npt.top
plumed partial_tempering 0.862 < npt.rest.top > replica03/npt.top
plumed partial_tempering 0.800 < npt.rest.top > replica04/npt.top
plumed partial_tempering 0.742 < npt.rest.top > replica05/npt.top
plumed partial_tempering 0.689 < npt.rest.top > replica06/npt.top
plumed partial_tempering 0.640 < npt.rest.top > replica07/npt.top
plumed partial_tempering 0.594 < npt.rest.top > replica08/npt.top
plumed partial_tempering 0.551 < npt.rest.top > replica09/npt.top
plumed partial_tempering 0.511 < npt.rest.top > replica10/npt.top


#### MD simulation

In [83]:
# Define the ensemble
presembl = "npt" # Previous ensemble
ensemble = "mds" # Ensemble

# Prepare the system to run the simulation
for i in range(1, replicas + 1):

    # Generate plumed file
    with open('%s/replica%02d/plumed.dat' % (basepath, i), 'w') as file:
        pass

    # Copy required files to the directory
    os.system("cp %s/%s.mdp %s/replica%02d/" % (basepath, ensemble, basepath, i))  
    os.system("cp %s/%s.gro %s/replica%02d/" % (basepath, presembl, basepath, i))

    # Prepare the system to run the simulation
    gmx(
        executable = "grompp",
        arguments = [
            "-maxwarn", "2"                              # Maximum number of warnings (there is a warning relative to charge)
        ],
        inputs = {
            "-f": "%s.mdp" % ensemble,                   # Minimization parameters file
            "-c": "%s.gro" % presembl,                   # Input structure file
            "-p": "%s.top" % presembl     # Topology file
        },
        outputs = {
            "-po": "%s.mdp" % ensemble,                  # Output parameters file
            "-pp": "%s.top" % ensemble,                 # Output topology file
            "-o" : "%s.tpr" % ensemble                   # Output portable xdr run input file
        },
        path = "%s/replica%02d" % (basepath, i)
    )

Setting the LD random seed to -10814006

Generated 1476621 of the 1476621 non-bonded parameter combinations

Generated 1476621 of the 1476621 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'SOP'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MET'

turning H bonds into constraints...

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
J. S. Hub, B. L. de Groot, H. Grubmueller, G. Groenhof
Quantifying Artifacts in Ewald Simulations of Inhomogeneous Systems with a Net
Charge
J. Chem. Theory Comput. 10 (2014) pp. 381-393
-------- -------- --- Thank You --- -------- --------

Analysing residue names:
There are:     1      Other residues
There are:   728    Protein residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing Protein...

The largest distance between excluded atoms is 0.408 nm between atom 20 and 22

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 293.

In [84]:
# Run the simulation
gmx(
    executable = "mdrun",
    ensemble = ensemble,
    arguments = [
        "-v",                                      # Verbose output
        "-plumed", "plumed.dat",                   # Plumed file
        "-deffnm", ensemble,                       # Input and output file name prefix
        "-multidir", "replica01", "replica02",     # Replica directories
        "replica03", "replica04", "replica05",
        "replica06", "replica07", "replica08",
        "replica09", "replica10", 
        "-replex","1000",
        "-hrex",
        "-cpt", "15",                              # Save state every 15 minutes
        "-cpo", "mds.cpt",                         # Checkpoint file
        "-dlb", "auto"                             # Dynamic load balancing auto
    ],
    inputs = {},
    outputs = {},
    scheduler = {"ntasks":"10",
                 "ntasks-per-node":"1"},
    name = compound,
    path = basepath, 
    wait = False
)

Submitted batch job 402


In [ ]:
# # Resume the simulation
# gmx(
#     executable = "mdrun",
#     ensemble = ensemble,
#     arguments = [
#         "-v",                                      # Verbose output
#         "-plumed", "plumed.dat",                   # Plumed file
#         "-deffnm", ensemble,                       # Input and output file name prefix
#         "-multidir", "replica01", "replica02",     # Replica directories
#         "replica03", "replica04", "replica05",
#         "replica06", "replica07", "replica08", 
#         "replica09", "replica10",
#         "-replex","1000",
#         "-hrex",
#         "-cpt", "15",                              # Save state every 15 minutes
#         "-cpo", "mds.cpt",                         # Checkpoint file
#     ],
#     inputs = {
#         "-cpi": "%s.cpt" % ensemble                # Resume from checkpoint file
#     },
#     outputs = {},
#     scheduler = {"ntasks":"10",
#                  "ntasks-per-node":"1"},
#     name = compound,
#     path = basepath, 
#     wait = False
# )

In [87]:
ensemble = "mds" # Ensemble

In [ ]:
# Print replica exchange rate
os.system('grep -A9 "average probabilities" replica01/mds.log')

In [88]:
# Generate the pressure xvg file
measure = 11     # Potential energy
sufix = "_e"

#Compute the the selected parameter from the ensemble simulation
for i in range(1, replicas + 1):
    gmx(
        executable = "energy",
        arguments = [],
        inputs = {
            "-f": "%s.edr" % ensemble
        },
        outputs = {
            "-o": "%s%s.xvg" % (ensemble, sufix)
        },
        stdin = "%d 0\n" % measure, 
        path = "%s/replica%02d/" % (basepath, i)
    )

# Generate the display the corresponding plots
xvg(title = "Molecular Dynamics Simulation",
    measure = "$\\lambda$",
    values = lambda_value,
    units="",
    replicas = replicas,
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Energy ($kJ\\cdot mol^{-1}$)",
    label = "Potential Energy",
    sufix = sufix,
    movavg = 100
)


Statistics over 10000001 steps [ 0.0000 through 10000.0000 ps ], 1 data sets
All statistics are over 100001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                  -7035.21          2    230.885   -2.71048  (kJ/mol)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol/replica01
Command line:
  gmx energy -f mds.edr -o mds_e.xvg

Opened mds.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-I

In [89]:
# Generate the pressure xvg file
measure = 15     # Temperature
sufix = "_t"

# Compute the the selected parameter from the ensemble simulation
for i in range(1, replicas + 1):
    gmx(
        executable = "energy",
        arguments = [],
        inputs = {
            "-f": "%s.edr" % ensemble
        },
        outputs = {
            "-o": "%s%s.xvg" % (ensemble, sufix)
        },
        stdin = "%d 0\n" % measure, 
        path = "%s/replica%02d/" % (basepath, i)
    )

# Generate the display the corresponding plots
xvg(title = "Molecular Dynamics Simulation",
    measure = "$\\lambda$",
    values = lambda_value,
    units="",
    replicas = replicas,
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Temperature (K)",
    label = "Temperature",
    sufix = sufix,
    movavg = 100
)


Statistics over 10000001 steps [ 0.0000 through 10000.0000 ps ], 1 data sets
All statistics are over 100001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 293.141       0.02    4.07802   0.125207  (K)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol/replica01
Command line:
  gmx energy -f mds.edr -o mds_t.xvg

Opened mds.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-Imp.-D

In [90]:
# Generate the pressure xvg file
measure = 17     # Pressure
sufix = "_p"

# Compute the the selected parameter from the ensemble simulation
for i in range(1, replicas + 1):
    gmx(
        executable = "energy",
        arguments = [],
        inputs = {
            "-f": "%s.edr" % ensemble
        },
        outputs = {
            "-o": "%s%s.xvg" % (ensemble, sufix)
        },
        stdin = "%d 0\n" % measure, 
        path = "%s/replica%02d/" % (basepath, i)
    )

# Generate the display the corresponding plots
xvg(title = "Molecular Dynamics Simulation",
    measure = "$\\lambda$",
    values = lambda_value,
    units="",
    replicas = replicas,
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Pressure (bar)",
    label = "Pressure",
    sufix = sufix,
    movavg = 100
)


Statistics over 10000001 steps [ 0.0000 through 10000.0000 ps ], 1 data sets
All statistics are over 100001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -2.05457        2.1    428.886   -1.68925  (bar)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol/replica01
Command line:
  gmx energy -f mds.edr -o mds_p.xvg

Opened mds.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-Imp.

In [91]:
# Generate the density xvg file
measure = 23     # Density
sufix = "_d"

# Compute the the selected parameter from the ensemble simulation
for i in range(1, replicas + 1):
    gmx(
        executable = "energy",
        arguments = [],
        inputs = {
            "-f": "%s.edr" % ensemble
        },
        outputs = {
            "-o": "%s%s.xvg" % (ensemble, sufix)
        },
        stdin = "%d 0\n" % measure, 
        path = "%s/replica%02d/" % (basepath, i)
    )

# Generate the display the corresponding plots
xvg(title = "Molecular Dynamics Simulation",
    measure = "$\\lambda$",
    values = lambda_value,
    units="",
    replicas = replicas,
    path = basepath,
    ensemble = ensemble,
    xlabel = "Time (ps)",
    ylabel = "Density ($Kg\\cdot m^{-3}$)",
    label = "Density",
    sufix = sufix,
    movavg = 100
)


Statistics over 10000001 steps [ 0.0000 through 10000.0000 ps ], 1 data sets
All statistics are over 100001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Density                     784.192       0.11    7.42974  -0.101312  (kg/m^3)

             :-) GROMACS - gmx energy, 2023.5-plumed_2.10.0_dev (-:

Executable:   /opt/gromacs-2023.5-plumed/bin/gmx
Data prefix:  /opt/gromacs-2023.5-plumed
Working dir:  /home/familia/research/synthetic_opioids/U48800/rest/methanol/replica01
Command line:
  gmx energy -f mds.edr -o mds_d.xvg

Opened mds.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Ryckaert-Bell.   4  Per.-I